<a href="https://colab.research.google.com/github/Sharma-Kamlesh/EVA5-Phase1/blob/main/S4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

In [10]:
class Net(nn.Module):
  def __init__(self):
    super(Net,  self).__init__()

    self.conv1  = nn.Conv2d(1,  16, 3,  padding=1,  bias=False)
    self.relu   = nn.ReLU()  
    self.conv2  = nn.Conv2d(16, 16, 3,  padding=1,  bias=False)
    self.pool  = nn.MaxPool2d(2,2)
    self.conv3  = nn.Conv2d(16, 16, 3,  padding=1,  bias=False)
    self.conv4  = nn.Conv2d(16, 32, 3)
    self.conv5  = nn.Conv2d(32,10,3)
    self.GAP  = nn.AvgPool2d(3)
    self.drop = nn.Dropout(0.25)
    #self.fc = nn.Linear(10,10)

  def forward(self, x):
    #x = self.pool(self.relu(self.conv2(self.relu(self.conv1(x)))))
    #x = self.GAP(self.conv4(self.pool(self.relu(self.conv3(x)))))
    #x = self.fc(x)
    x=self.relu(self.conv1(x))      # 1X28X28 | 3X3X1X16 | 16X28X28
    x= self.drop(x)
    x=self.relu(self.conv2(x))      # 16X28X28 | 3X3X16X16 | 16X28X28
    x= self.drop(x)
    x=self.pool(x)                  # 16X28X28 |  /2  | 16X14X14
    x=self.relu(self.conv3(x))      # 16X14X14 | 3X3X16X16 | 16X14X14
    x= self.drop(x)
    x=self.relu(self.conv4(x))      # 16X14X14 | 3X3X16X32 | 32X12X12
    x= self.drop(x)
    x=self.pool(x)                  # 32X12X12 | /2 | 32X6X6
    x=self.conv5(x)                 # 32X6X6  | 3X3X32X10 | 10X4X4
   
    x=self.GAP(x)                   # 10X4X4  |  -3 | 10X1x1
    x = x.view(-1,10)
    return F.log_softmax(x, dim=1)

In [11]:

!pip install torchsummary
from torchsummary import summary
use_cuda  = torch.cuda.is_available()
device  = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
           Dropout-3           [-1, 16, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           2,304
              ReLU-5           [-1, 16, 28, 28]               0
           Dropout-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,304
              ReLU-9           [-1, 16, 14, 14]               0
          Dropout-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 32, 12, 12]           4,640
             ReLU-12           [-1, 32, 12, 12]               0
        MaxPool2d-13             [-1, 32, 6, 6]               0
           Conv2d-14             [-1, 1

In [4]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                       
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                      
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader,  optimizer,  epoch):
  model.train()
  pbar  = tqdm(train_loader)
  for batch_idx,  (data,  target) in enumerate(pbar):
    data, target  = data.to(device),  target.to(device)
    optimizer.zero_grad()
    output  = model(data)
    loss  = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    pbar.set_description(desc=  f'loss={loss.item()} batch_id={batch_idx}')

def test(model, device, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data,target in test_loader:
      data,target = data.to(device),  target.to(device)
      output  = model(data)
      test_loss +=  F.nll_loss(output,  target, reduction='sum').item()
      pred  = output.argmax(dim=1, keepdim=True)
      correct +=  pred.eq(target.view_as(pred)).sum().item()
  test_loss /=  len(test_loader.dataset)

  print('\nTest set: Average loss:  {:.4f}, Accuracy: {}/{} ({:.2f}%\n'.format(
      test_loss,  correct,  len(test_loader.dataset),
      100.* correct / len(test_loader.dataset)
  ))

In [6]:
model = Net().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss:  0.0979, Accuracy: 9731/10000 (97.31%



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss:  0.0714, Accuracy: 9823/10000 (98.23%



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss:  0.0282, Accuracy: 9921/10000 (99.21%



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss:  0.0270, Accuracy: 9921/10000 (99.21%



loss=0.03077799640595913 batch_id=937: 100%|██████████| 938/938 [00:16<00:00, 57.52it/s]



Test set: Average loss:  0.0254, Accuracy: 9928/10000 (99.28%

